In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# ARC Аркадий Соловьев
# Скрипт преобразования файлов аннотаций для ресайза кадров имеющих размер 1920 на 1080
# в размер кадра 640 на 640 с верхней и нижней черными полосами

# Указать путь к папке с исходными аннотациями
labels_folder = "/content/drive/MyDrive/1 Исходные файлы для Датасета 4.0/medium/train/labels"
# Указать путь к папке для новых аннотаций
new_labels_folder = "/content/drive/MyDrive/1 Датасет 4.0/medium/train/labels"

# Размеры входного кадра
input_width, input_height = 1920, 1080

# Размеры выходного кадра 640 на  640 при резайзе с сохранением пропорций
# Рассчитываем пропорциональное изменение размера
output_width = 640
output_height = int(input_height * output_width / input_width)

# Создать новую папку, если она не существует и удалить, если существует
if os.path.exists(new_labels_folder): # Если папка существует
    # Получаем список файлов в папке
    file_list = os.listdir(new_labels_folder)
    # Проходим по каждому файлу и удаляем его
    for file_name in file_list:
        file_path = os.path.join(new_labels_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)           # Удалить файл
    os.rmdir(new_labels_folder)            # удалить папку
if not os.path.exists(new_labels_folder):  # Если папка не существует
    os.makedirs(new_labels_folder)         # создать папку

# Обработка каждого файла аннотаций
for filename in os.listdir(labels_folder):  # Проход по всем файлам папки
    if filename.endswith(".txt"):           # Проверка окончания имени файла
        file_path = os.path.join(labels_folder, filename) # Путь к файлу
        new_file_prefix = os.path.splitext(filename)[0]   # Скопировать и сохранить имя исходного файла

        with open(file_path, "r") as file:  # Открыть файл для чтения
            annotations = file.readlines()  # Читать строки (аннотации)
    class_n = []    # Список номеров классов
    polygons = []   # Список новых полигонов для кадра 640 на 640
    for i, annotation in enumerate(annotations):  # Проход по всем строкам (полигонам) файла аннотаций
        annotation = annotation.strip()           # Убрать начальные и оконечные пробелы в строке
        # Прочитать имя класса и координаты
        class_id, *coordinates = map(float, annotation.split())
        class_n.append(class_id)   # Заполнение списка номеров классов
        # Координаты x y полигонов
        polygon_coords = [(coordinates[j], coordinates[j + 1]) for j in range(0, len(coordinates), 2)]

        new_coords = [] # Список новых координат вершин полигонов в долях кадра 640 на 640 с черными полосами после ресайза
        for coord in polygon_coords:    # Проходим по координатам
            x_new = coord[0]   # Координаты x в долях кадра 640 на 640 не изменились
            y_new = ((coord[1] * 360) + 140) / 640  # Пересчет по Y в долях кадра 640 на 640
            # output_height = 360 высота кадра после ресайза при ширине кадра 640, 140 - ширина черной полосы сверху и снизу кадра
            new_coords.append(f"{x_new} {y_new}") # Координаты в долях кадра 640 на 640
        polygons.append(' '.join(new_coords)) # Запись строк (полигонов)

    # Запись полигонов в файлы аннотации с расширением "_0.txt", resize кадр
    for i, polygon in enumerate(polygons):    # Проходим по строкам (полигонам)
        new_file_path = os.path.join(new_labels_folder, f"{new_file_prefix}_0.txt") # Формирование пути к записываемым файлам
        with open(new_file_path, "a") as new_file:               # Открыть файл для записи
            new_file.write(f"{int(class_n[i])} {polygon}\n")     # Запись файла
